In [86]:
import whisper
import os
import ssl
import certifi

ssl_context = ssl.create_default_context(cafile=certifi.where())

def transcribe_audio(file_path, model_size="medium", language="es"):
    model = whisper.load_model(model_size)
    result = model.transcribe(file_path, language=language)
    return result["text"]

def save_transcript(text, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)

if __name__ == "__main__":
    audio_path = input("Путь к локальному аудиофайлу (.mp3, .wav и др.): ").strip()

    if not os.path.isfile(audio_path):
        print(f"Файл не найден: {audio_path}")
        exit(1)

    print("🧠 Распознаём текст (whisper)...")
    transcript = transcribe_audio(audio_path, model_size="medium", language="es")

    output_file = audio_path.rsplit(".", 1)[0] + ".txt"
    save_transcript(transcript, output_file)

    print(f"\n✅ Готово! Транскрипт сохранён в {output_file}")
    print("🔎 Первые 300 символов:\n")
    print(transcript[:300])

🧠 Распознаём текст (whisper)...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



✅ Готово! Транскрипт сохранён в /Users/elizaveta/Documents/Проект_2/7septimo.txt
🔎 Первые 300 символов:

 Esta es la historia de Lola y Ana, quien comparten un piso en Barcelona, y de sus vecinos de en frente Pablo y Sam el americano. Lola quiere ser rica, Pablo quiere ser actor y Sam quiere invitar a Ana al cine. Prepárate para el próximo episodio de Extra. No puedo creer que vuelvas de Argentina. ¿Re


In [3]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from nltk import word_tokenize, bigrams

nlp = spacy.load("es_core_news_sm")

def load_and_split_sentences(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()], text

def extract_pos_filtered_words(text, allowed_pos={"NOUN", "VERB", "ADJ", "ADV"}):
    doc = nlp(text)
    return [token.text.lower() for token in doc 
            if token.pos_ in allowed_pos and not token.is_stop and token.is_alpha]

def extract_keywords_by_pos(text, allowed_pos={"NOUN", "VERB", "ADJ", "ADV", "PREP"}, top_n=20):
    words = extract_pos_filtered_words(text, allowed_pos)
    filtered_text = " ".join(words)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([filtered_text])
    scores = zip(vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0])
    sorted_keywords = sorted(scores, key=lambda x: x[1], reverse=True)
    
    return [word for word, _ in sorted_keywords[:top_n]]

def extract_top_bigrams(text, top_n=20, exclude_words=None):
    if exclude_words is None:
        exclude_words = set()

    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha()]  # Только алфавитные слова

    bigram_list = list(bigrams(tokens))

    # Фильтрация биграм, содержащих исключённые слова
    filtered_bigrams = [
        pair for pair in bigram_list
        if pair[0] not in exclude_words and pair[1] not in exclude_words
    ]

    bigram_freq = Counter(filtered_bigrams)
    top_bigrams = [" ".join(pair) for pair, _ in bigram_freq.most_common(top_n)]
    return top_bigrams


# Загрузка текста
sentences, text = load_and_split_sentences("1primero.txt")

# Ключевые слова по POS
keywords = extract_keywords_by_pos(text, top_n=50)

# Частотные биграммы
exclude_names = {"ana", "lola", "sam", "pablo", "antonio"}
bigrams = extract_top_bigrams(text, top_n=50, exclude_words=exclude_names)

# Для вывода:
print("Ключевые слова (TF-IDF по NOUN/VERB/ADJ/ADV):")
print(keywords)

print("\nЧастотные биграммы:")
print(bigrams)

Ключевые слова (TF-IDF по NOUN/VERB/ADJ/ADV):
['horno', 'español', 'quieres', 'llamo', 'perro', 'museo', 'americano', 'coches', 'chicas', 'vivo', 'correo', 'dormitorio', 'ducha', 'estupendo', 'estás', 'quedar', 'tienes', 'cama', 'carta', 'chica', 'chico', 'compras', 'dije', 'duerme', 'factura', 'gracias', 'hablar', 'queda', 'rápido', 'tomar', 'abajo', 'acabó', 'amigos', 'años', 'baño', 'bici', 'bicicleta', 'celebrarlo', 'corresponsal', 'dices', 'digas', 'dormir', 'echo', 'favor', 'febrero', 'fuertes', 'gusta', 'hablo', 'hombres', 'importa']

Частотные биграммы:
['el horno', 'en el', 'el perro', 'me llamo', 'un museo', 'está en', 'de américa', 'no no', 'en un', 'perro está', 'esto es', 'muy bien', 'es la', 'el correo', 'la factura', 'bien el', 'tomar algo', 'es un', 'quiere decir', 'con coches', 'es el', 'se vaya', 'un poco', 'perro de', 'duerme en', 'de compras', 'compras para', 'en españa', 'le gusta', 'están las', 'una carta', 'te lo', 'se acabó', 'y no', 'no me', 'lo siento', 'luis 